# ItaData2024
modal example

In [1]:
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random

### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [ ]:
nan_replacer!(x::AbstractArray{Float64}) = replace!(x, NaN => 0.0)

function afe(x::AbstractVector{Float64}; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = 8000
    norm = true
    speech_detection = false
    # stft module
    stft_length = 256
    win_type = (:hann, :periodic)
    win_length = 256
    overlap_length = 128
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = 26
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = (300, round(Int, sr / 2))
    # mel spectrogram module
    db_scale = false
    # mfcc module
    ncoeffs = 13
    rectification = :log                    # :log, :cubic_root
    dither = true
    # f0 module
    method = :nfc
    f0_range = (50, 400)

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        stft_length=stft_length,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # mfcc module
    mfcc = get_mfcc(
        source=melspec,
        ncoeffs=ncoeffs,
        rectification=rectification,
        dither=dither,
    );

    # f0 module
    f0 = get_f0(
        source=stftspec,
        method=method,
        freq_range=f0_range
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    x_features = hcat(
        melspec.spec',
        mfcc.mfcc',
        f0.f0,
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );

    nan_replacer!(x_features)

    return x_features
end

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"
variable_names = [
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:26]...,
    ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:13]...,
    "\e[$(color_code[:green])mf0\e[0m", "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
]

X = DataFrame([name => Vector{Float64}[] for name in [match(r_select, v)[1] for v in variable_names]])

features = [minimum, maximum]

for i in 1:nrow(x)
    push!(X, collect(eachcol(afe(x[i, :audio]))))
end

yc = CategoricalArray(y);

### Data compression for modal analysis

In [5]:
train_ratio = 0.8
train, test = partition(eachindex(yc), train_ratio, shuffle=true)
# train, test = partition(eachindex(yc), train_ratio, shuffle=false) ### Debug
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 51) - 403
Test set size: (101, 51) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    model = ModalDecisionTree(; relations = :IA7, features = features)
    mach = machine(model, X_train, y_train) |> fit!
end

report(learned_dt_tree).printmodel(variable_names_map=variable_names);

┌ Info: Precomputing logiset...
└ @ ModalDecisionTrees.MLJInterface /home/paso/.julia/dev/ModalDecisionTrees.jl/src/interfaces/MLJ/wrapdataset.jl:135
┌ Info: Training machine(ModalDecisionTree(max_depth = nothing, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


▣ ⟨G⟩(min[mfcc3] < -0.42238800051820113)
├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩(min[cntrd] ≥ 1241.5325730780178))
│├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩(min[sprd] ≥ 693.3580008260416)))
││├✔ Healthy
││└✘ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨AO⟩(min[mel3=488Hz] ≥ 2.773555451829634e-5)))
││ ├✔ Pneumonia
││ └✘ Healthy
│└✘ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩(min[mfcc9] ≥ 0.7471857551919938))
│ ├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[mfcc9] ≥ 0.7471857551919938) ∧ ⟨G⟩(min[mfcc11] < -0.6418413457274044)))
│ │├✔ Healthy
│ │└✘ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[mfcc9] ≥ 0.7471857551919938) ∧ (min[mel4=558Hz] ≥ 7.914865476662152e-5)))
│ │ ├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[mfcc9] ≥ 0.7471857551919938) ∧ (min[mel4=558Hz] ≥ 7.914865476662152e-5) ∧ (min[mel3=488Hz] ≥ 0.005262215103371913)))
│ │ │├✔ Healthy
│ │ │└✘

### Model inspection & rule study

In [7]:
_, mtree = report(mach).sprinkle(X_test, y_test)
sole_dt = ModalDecisionTrees.translate(mtree)

printmodel(sole_dt; show_metrics = true, variable_names_map=variable_names);

Applying tree... 100%|███████████████████████████████████| Time: 0:00:06


▣ ⟨G⟩(min[mfcc3] < -0.42238800051820113)
├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩(min[cntrd] ≥ 1241.5325730780178))
│├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩(min[sprd] ≥ 693.3580008260416)))
││├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩((min[sprd] ≥ 693.3580008260416) ∧ (min[mfcc2] ≥ 2.132583349861581))))
│││├✔ Healthy : (ninstances = 27, ncovered = 27, confidence = 0.89, lift = 1.0)
│││└✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││└✘ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨AO⟩(min[mel3=488Hz] ≥ 2.773555451829634e-5)))
││ ├✔ Pneumonia : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││ └✘ Healthy : (ninstances = 3, ncovered = 3, confidence = 0.67, lift = 1.0)
│└✘ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩(min[mfcc9] ≥ 0.7471857551919938))
│ ├✔ ⟨G⟩((min[mfcc3] < -0.42238800051820113) 

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩((min[sprd] ≥ 693.3580008260416) ∧ (min[mfcc2] ≥ 2.132583349861581))))  ↣  Healthy : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 0.89, lift = 2.0, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩(min[sprd] ≥ 693.3580008260416))) ∧ [G]((min[mfcc3] < -0.42238800051820113) → [A̅O̅]((min[cntrd] ≥ 1241.5325730780178) → [L]((min[sprd] ≥ 693.3580008260416) → (min[mfcc2] < 2.132583349861581))))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.24, natoms = 7)
▣ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨AO⟩(min[mel3=488Hz] ≥ 2.773555451829634e-5))) ∧ [G]((min[mfcc3] < -0.42238800051820113) → [A̅O̅]((min[cntrd] ≥ 1241.5325730780178) → [L](min[sprd] < 693.3580008260416)))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.42) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.53) ∧ ⟨L⟩((min[sprd] ≥ 693.36) ∧ (min[mfcc2] ≥ 2.13))))  ↣  Healthy : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 0.89, lift = 2.0, natoms = 4)
▣ ⟨G⟩((min[mfcc3] < -0.42) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.53) ∧ ⟨L⟩(min[sprd] ≥ 693.36))) ∧ [G]((min[mfcc3] < -0.42) → [A̅O̅]((min[cntrd] ≥ 1241.53) → [L]((min[sprd] ≥ 693.36) → (min[mfcc2] < 2.13))))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.24, natoms = 7)
▣ ⟨G⟩((min[mfcc3] < -0.42) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.53) ∧ ⟨AO⟩(min[mel3=488Hz] ≥ 0.0))) ∧ [G]((min[mfcc3] < -0.42) → [A̅O̅]((min[cntrd] ≥ 1241.53) → [L](min[sprd] < 693.36)))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.8, natoms = 6)
▣ ⟨G⟩((min[mfcc3] < -0.42) ∧ ⟨A̅O̅⟩(min[cntrd] ≥ 1241.53)) ∧ [G]((min[mfcc3] < -0.42) → [A̅O̅]((min[cntrd] ≥ 1241.53) → [L](min[sprd] < 693.36))) ∧ [G]((min[mfcc3] < -0.42) → [A̅O̅]((min[cnt

### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

13-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 27, coverage = 0.26732673267326734, confidence = 0.8888888888888888, lift = 1.9950617283950616, natoms = 4)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.2444444444444445, natoms = 7)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8035714285714284, natoms = 6)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 0.6666666666666666, lift = 1.4962962962962962, natoms = 8)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 2.2444444444444445, natoms = 9)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 0.6666666666666666, lift = 1.4962962962962962, natoms = 11)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence 

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map=variable_names);

▣ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨AO⟩(min[mel3=488Hz] ≥ 2.773555451829634e-5))) ∧ [G]((min[mfcc3] < -0.42238800051820113) → [A̅O̅]((min[cntrd] ≥ 1241.5325730780178) → [L](min[sprd] < 693.3580008260416)))  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8035714285714284, natoms = 6, height = 6)
▣ ⟨G⟩((min[mfcc2] < 1.4408221727109625) ∧ (min[mel1=359Hz] ≥ 2.2117528019512427e-5)) ∧ [G](min[mfcc3] ≥ -0.42238800051820113)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.2444444444444445, natoms = 3, height = 3)
▣ ⟨G⟩((min[mfcc3] < -0.42238800051820113) ∧ ⟨A̅O̅⟩((min[cntrd] ≥ 1241.5325730780178) ∧ ⟨L⟩(min[sprd] ≥ 693.3580008260416))) ∧ [G]((min[mfcc3] < -0.42238800051820113) → [A̅O̅]((min[cntrd] ≥ 1241.5325730780178) → [L]((min[sprd] ≥ 693.3580008260416) → (min[mfcc2] < 2.132583349861581))))  ↣  Healthy : (ninstances = 101

### Pretty table of rules and their metrics

In [13]:
metricstable(interesting_rules; variable_names_map=variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                                                                                                                                                